In [ ]:
from github import Github as pg
from github import RateLimitExceededException, BadCredentialsException, BadAttributeException, GithubException, UnknownObjectException, BadUserAgentException
import csv
import requests #retries and timeout Github Exception handling
import time #sleep the process for sometime

NUM_TOKENS = 2

#using the personal access token
access_token1 = <add_token_here>
access_token2 = <add_token_here>

# Set initial API token
api_token_queue = [access_token1,access_token2]

token_i = 0
g = pg(api_token_queue[token_i], retry =3)

In [ ]:
#search repositories: langauge as a criteria
#"HCL", "TypeScript", "Lua", "Go", "Rust", "Shell", "Makefile", "C", "Kotlin", "Python"
languages = [ "HCL", "Rust", "TypeScript", "Lua", "Go", "Shell", "Makefile", "C", "Kotlin", "Python"] 
for lang in languages:
    this_rate= str(g.get_rate_limit())
    this_rate= int(this_rate.split(",")[1].split("=")[1])
    if(this_rate <= 2000):
        token_i =(token_i + 1) % NUM_TOKENS
    while True:
        try:
            g = pg(api_token_queue[token_i], retry =3)
            repositories = g.search_repositories(query="language:"+lang, sort='stars', order='desc')
            with open("../Output/"+lang+".csv", "w", newline="") as f:
                writer = csv.writer(f)
                writer.writerow(["Repo Name", "Stargazer_Count"])
                for repo in repositories[:200]:
                    while True:
                            try: 
                                #Print repo and star of it.
                                repo_nm = str(repo).split('"')[1]
                                repo_str = repo.stargazers_count
                    #             print("Repo", repo_nm, "has #star =", repo_str)
                                #Save it in csv with filename as langauge.csv
                                writer.writerow([repo_nm, repo_str])
                            except RateLimitExceededException as e:
                                print(e.status)
                                print('Rate Limit exceeded')
                                time.sleep(300) #5 minutes
                                continue
                            except BadCredentialsException as e:
                                print(e.status)
                                print("Bad Credentials exception")
                                break
                            except UnknownObjectException as e:
                                print(e.status)
                                print("The repository doesn't exist anymore")
                                break
                            except GithubException as e:
                                print(e.status)
                                print("Github General Exception")
                                break
                            except requests.exceptions.ConnectionError as e:
                                print("Retries limit exceeded")
                                print(str(e))
                                time.sleep(10)
                                continue
                            except requests.exceptions.Timeout as e:
                                print("Timeout exception")
                                print(str(e))
                                time.sleep(10)
                                continue
                            break
        except RateLimitExceededException as e:
            print(e.status)
            print('Rate Limit exceeded')
            time.sleep(300) #5 minutes
            continue
        except BadCredentialsException as e:
            print(e.status)
            print("Bad Credentials exception")
            break
        except UnknownObjectException as e:
            print(e.status)
            print("The repository doesn't exist anymore")
            break
        except GithubException as e:
            print(e.status)
            print("Github General Exception")
            break
        except requests.exceptions.ConnectionError as e:
            print("Retries limit exceeded")
            print(str(e))
            time.sleep(10)
            continue
        except requests.exceptions.Timeout as e:
            print("Timeout exception")
            print(str(e))
            time.sleep(10)
            continue
        break